In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
from pyspark.sql import types as T

schema = T.StructType([
    T.StructField("id", T.IntegerType(), False),
    T.StructField("name", T.StringType(), False),
    T.StructField("partition_id", T.IntegerType(), False)
])

data = [
    (1, "A", 1),
    (2, "B", 1),
    (3, "C", 1),
    (4, "D", 2),
    (5, "E", 2),
    (6, "F", 2),
    (7, "G", 3),
    (8, "H", 3),
    (9, "I", 3)
]

(spark
 .createDataFrame(data, schema)
 .repartition(3, "partition_id")
 .rdd
 .glom()
 .collect())

[[Row(id=4, name='D', partition_id=2),
  Row(id=5, name='E', partition_id=2),
  Row(id=6, name='F', partition_id=2),
  Row(id=7, name='G', partition_id=3),
  Row(id=8, name='H', partition_id=3),
  Row(id=9, name='I', partition_id=3)],
 [Row(id=1, name='A', partition_id=1),
  Row(id=2, name='B', partition_id=1),
  Row(id=3, name='C', partition_id=1)],
 []]

In [5]:
(spark
 .createDataFrame(data, schema)
 .rdd
 .keyBy(lambda x: x.partition_id)
 .partitionBy(3, int)
 .glom()
 .collect())

[[(3, Row(id=7, name='G', partition_id=3)),
  (3, Row(id=8, name='H', partition_id=3)),
  (3, Row(id=9, name='I', partition_id=3))],
 [(1, Row(id=1, name='A', partition_id=1)),
  (1, Row(id=2, name='B', partition_id=1)),
  (1, Row(id=3, name='C', partition_id=1))],
 [(2, Row(id=4, name='D', partition_id=2)),
  (2, Row(id=5, name='E', partition_id=2)),
  (2, Row(id=6, name='F', partition_id=2))]]